In [1]:
%reload_ext autoreload
%autoreload 2


import matplotlib.pyplot as plt
import os
import sys
sys.path.append(os.path.abspath(os.path.join('..')))

from util.fusion_util import *
from sites.config import *
from model.pdr import PDR

ModuleNotFoundError: No module named 'fastdtw'

In [5]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [21]:
site = MtrCtl()
site_name = site.name
map_constraints = site.in_constraints, site.out_constraints
map_path = os.path.join('..',site.map_path)
dataset_dir = os.path.join('..', site.dataset_dir)
log_dir = os.path.join('..', site.log_dir)

In [18]:
data = read_fusion_data(dataset_dir)
data_idx = 3

../../data/mtr/ctl/naturally_Mtr_Ctl_5572a8422e8a67cf_134_621_157_621_1596542397047_WiFi.txt
../../data/mtr/ctl/naturally_Mtr_Ctl_5572a8422e8a67cf_135_623_314_736_1596542195849_WiFi.txt
../../data/mtr/ctl/naturally_Mtr_Ctl_5572a8422e8a67cf_409_684_327_744_1596542067235_WiFi.txt
../../data/mtr/ctl/naturally_Mtr_Ctl_b59a361f6b3e7f11_148_602_389_689_1597138057556_WiFi.txt


In [36]:
pdr = PDR(site.scale)
locs = []
truths = []
x, y = data[data_idx][0]['loc']
theta = 330 / 180 * np.pi

for d in data[data_idx]:
    dl = pdr.get_travel_distance(d['acc'])
    heading, _, _, _ = pdr.get_orientation(np.mean(d['mag'], axis=0), np.mean(d['gravity'], axis=0))
#     print(heading * 180 / np.pi % 360)

    x += np.cos(heading + theta) * dl
    y += np.sin(heading + theta) * dl

    locs.append((x, y))
    truths.append(d['loc'])

-1.5707963267948966


In [37]:
(240+330)%360

210

In [35]:
%matplotlib widget

im = plt.imread(map_path)
plt.figure()
plt.imshow(im)
plt.scatter(*zip(*locs), s=1)
plt.scatter(*zip(*truths), c='r', s=1)
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …